In [1]:
import pandas as pd
import pickle
import gensim
from gensim import corpora

import spacy
from spacy.lang.en import English

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation

import random

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/katieta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dem_tweets = pd.read_csv('../data/dem_tweets_cleaned.csv')
rep_tweets = pd.read_csv('../data/rep_tweets_cleaned.csv')
dem_sampled_tweets = pd.read_csv('../data/dem_sampled_tweets.csv')
rep_sampled_tweets = pd.read_csv('../data/rep_sampled_tweets.csv')

FileNotFoundError: File b'../data/dem_tweets_cleaned.csv' does not exist

In [ ]:
rep_corpus = rep_tweets['stripped_text'].values
dem_corpus = dem_tweets['stripped_text'].values

In [ ]:
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
#         elif token.orth_.startswith('#'):
#             lda_tokens.append('HASHTAG')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [ ]:
# remove common words and tokenize
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1
# print(stoplist)

In [3]:
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

[nltk_data] Downloading package wordnet to /Users/katieta/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/katieta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [6]:
def get_text_data(corpus, full_data=False):
    text_data = []
    for i in range(len(corpus)):
        if i % 20000 == 0:
            print("{} / {}".format(i, len(corpus)))
        try:
            tokens = prepare_text_for_lda(corpus[i])
            if random.random() > .90 and full_data == False:
                text_data.append(tokens)
            if full_data:
                text_data.append(tokens)
        except Exception as e:
            print(e)
    print("finished")
    return text_data

## Gensim Topic Modeling

In [7]:
def get_dict_corpus(text_data):
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]
    return dictionary, corpus

def run_topic_model(text_data, num_topics, num_passes, num_words, model_name, save=True):
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]
    
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = num_topics, id2word=dictionary, passes=num_passes)
    if save:
        ldamodel.save(model_name + '.gensim')
    topics = ldamodel.print_topics(num_words=num_words)
    return topics

In [12]:
# dem_text_data = get_text_data(dem_corpus)
# pickle.dump(dem_text_data, open('dem_text_data.pkl', 'wb'))

# with open('pickles/dem_text_data.pkl', 'rb') as f:
#     dem_text_data = pickle.loads(f.read())

NameError: name 'dem_corpus' is not defined

In [9]:
topics = run_topic_model(text_data=dem_text_data
                         , num_topics=15
                         , num_passes=15
                         , num_words=4
                         , model_name='dem_model_15')

for topic in topics:
    print(topic)

NameError: name 'dem_text_data' is not defined

In [35]:
idx = []
idx_retweets = []
for i, r in dem_tweets.iterrows():
#     print(r['media'], len(r['media']))
    if len(r['media']) == 2:
        idx.append(i)
    else:
        idx_retweets.append(i)


In [34]:
print(len(idx))

138230


In [11]:
# tweets with photos
dem_tweets[dem_tweets['media'].str.contains('pic.twitter')].sample(100)

,candidate,twitter_handle,date,original_text,stripped_text,retweets,favorites,geo,mentions,hashtags,media
242004,Emily Martz,FlipNY45,2018-03-06 12:42,Get excited! # NY21pic.twitter.com/2Bc5EF4ZSi,Get excited! #NY21,1,8,NaN,[],['#NY21'],['pic.twitter.com/2Bc5EF4ZSi']
208759,Debra Haaland,Deb4CongressNM,2018-07-25 20:00,I'm so honored to have the support of the @ Gi...,I'm so honored to have the support of the @Gif...,62,192,NaN,['@GiffordsCourage.'],[],['pic.twitter.com/1WtdJs1pR9']
230421,Omar Vaid,omarvaid,2018-06-21 23:48,"Thank you @ IrishMadonna718, one of my favorit...","Thank you @IrishMadonna718, one of my favorite...",1,11,NaN,"['@IrishMadonna718,']",[],['pic.twitter.com/UjdvF1NcQy']
197056,Jane Raybould,JaneRaybould,2018-06-29 19:32,Packed house tonight! Amazing to see the energ...,Packed house tonight! Amazing to see the energ...,11,63,NaN,[],"['#TeamRaybould', '#nesen']",['pic.twitter.com/wF6z2tA5Yw']
124496,Brent Welder,BrentWelder,2018-06-25 10:18,Across the country & right here in Kansas ther...,Across the country & right here in Kansas ther...,9,13,NaN,[],"['#ksleg', '#votingrights', '#protectourvote',...",['pic.twitter.com/TfmlV3rpme']
123214,Laura Kelly,LauraKellyKS,2018-08-20 15:27,Want to help make sure Laura Kelly gets electe...,Want to help make sure Laura Kelly gets electe...,56,92,NaN,[],['#TeamKelly!'],['pic.twitter.com/P8x5xrzT1s']
123548,Laura Kelly,LauraKellyKS,2018-03-09 13:07,Laura is the candidate with the right combinat...,Laura is the candidate with the right combinat...,7,13,NaN,[],[],"['pic.twitter.com/zyC0YSxTn5', 'http://bit.ly/..."
132602,Patti Piatt,PiattPatti,2018-04-10 09:20,AAUW- Women will have wait 100 years for women...,AAUW- Women will have wait 100 years for women...,2,5,NaN,[],"['#electwomen', '#EqualPayDay']","['pic.twitter.com/kh36cpN3lw""']"
242301,Katie Wilson,AdirondacKatie,2018-06-13 17:46,Hittin the airwaves in # watertownny https://i...,Hittin the airwaves in #watertownny,1,8,NaN,[],['#watertownny'],"['pic.twitter.com/VF8i27d2BK', 'https://ift.tt..."
62782,Donna Lynne,DonnaLynneCO,2018-06-17 11:32,HAPPY # PRIDE pic.twitter.com/bCmJhqJUH7 â b...,HAPPY #PRIDE bei Cheesman Park,4,6,NaN,[],['#PRIDE'],['pic.twitter.com/bCmJhqJUH7']


In [36]:
# retweets
dem_tweets.iloc[idx_retweets,:]

,candidate,twitter_handle,date,original_text,stripped_text,retweets,favorites,geo,mentions,hashtags,media
0,Christopher Countryman,CA_Countryman,2018-11-05 13:53,With corruption being one of the biggest conce...,With corruption being one of the biggest conce...,1,1,NaN,[],[],['https://www.al.com/election/2018/10/corrupti...
1,Christopher Countryman,CA_Countryman,2018-11-05 13:50,With corruption being one of the biggest conce...,With corruption being one of the biggest conce...,0,1,NaN,[],[],['https://www.facebook.com/329505890581395/pos...
2,Christopher Countryman,CA_Countryman,2018-11-05 10:49,Here is an important notice concerning Christo...,Here is an important notice concerning Christo...,0,0,NaN,[],[],['https://www.facebook.com/329505890581395/pos...
3,Christopher Countryman,CA_Countryman,2018-11-04 07:19,I believe that we need to find real solutions ...,I believe that we need to find real solutions ...,0,0,NaN,[],[],['https://www.facebook.com/329505890581395/pos...
4,Christopher Countryman,CA_Countryman,2018-11-02 16:15,"It's time to turn the page on Alabama's past, ...","It's time to turn the page on Alabama's past, ...",0,0,NaN,[],[],['https://www.facebook.com/1753652668215749/po...
5,Christopher Countryman,CA_Countryman,2018-11-02 00:25,There is hope for Alabama. This inspiring vide...,There is hope for Alabama. This inspiring vide...,1,1,NaN,[],[],['https://www.facebook.com/rethinkalabama/vide...
6,Christopher Countryman,CA_Countryman,2018-11-01 22:17,Live https://www.facebook.com/rethinkalabama/v...,Live,0,0,NaN,[],[],['https://www.facebook.com/rethinkalabama/vide...
7,Christopher Countryman,CA_Countryman,2018-11-01 13:18,Join us for a special and important webcast co...,Join us for a special and important webcast co...,0,0,NaN,[],[],['https://www.facebook.com/1753652668215749/po...
8,Christopher Countryman,CA_Countryman,2018-10-31 09:34,Countryman isn't standing down...his fight con...,Countryman isn't standing down...his fight con...,0,0,NaN,[],[],['https://www.facebook.com/rethinkalabama/post...
9,Christopher Countryman,CA_Countryman,2018-10-30 16:09,Please join us for a very special 2018 Alabama...,Please join us for a very special 2018 Alabama...,0,0,NaN,[],[],['https://www.facebook.com/1753652668215749/po...


### Only use tweets with > 4 words

In [14]:
print(len(dem_tweets[dem_tweets['stripped_text'].apply(lambda x: len(x.split(' ')) <= 4)]))
dem_long_tweets = dem_tweets[dem_tweets['stripped_text'].apply(lambda x: len(x.split(' ')) > 4)]

# dem_text_data_long = get_text_data(dem_long_tweets['stripped_text'].values)
pickle.dump(dem_text_data_long, open('pickles/dem_text_data_long.pkl', 'wb'))

# with open('dem_text_data.pkl', 'rb') as f:
#     dem_text_data = dem_text_data.loads(f.read())

In [124]:
print(len(dem_text_data_long))
c = 1
for d in dem_text_data_long:
    if 'thank' in d:
        c += 1
print(c)

146417
10795


In [17]:
# BEST RESULTS, takes some time though

topics = run_topic_model(text_data=dem_text_data_long
                         , num_topics=15
                         , num_passes=15
                         , num_words=4
                         , model_name='dem_model_15')

for topic in topics:
    print(topic)

(0, '0.481*"SCREEN_NAME" + 0.015*"bluewave2018" + 0.013*"endorse" + 0.012*"attack"')
(1, '0.056*"voter" + 0.032*"candidate" + 0.030*"election" + 0.027*"november"')
(2, '0.031*"politician" + 0.021*"court" + 0.020*"result" + 0.020*"spread"')
(3, '0.058*"texas" + 0.049*"today" + 0.049*"friend" + 0.022*"yesterday"')
(4, '0.048*"school" + 0.042*"public" + 0.030*"teacher" + 0.027*"education"')
(5, '0.038*"would" + 0.021*"social" + 0.020*"security" + 0.019*"trump"')
(6, '0.036*"worker" + 0.034*"union" + 0.030*"business" + 0.024*"american"')
(7, '0.156*"support" + 0.103*"thank" + 0.041*"campaign" + 0.039*"going"')
(8, '0.074*"people" + 0.030*"congress" + 0.028*"running" + 0.022*"working"')
(9, '0.036*"virginia" + 0.036*"veteran" + 0.026*"really" + 0.021*"epidemic"')
(10, '0.041*"healthcare" + 0.040*"right" + 0.032*"family" + 0.018*"child"')
(11, '0.060*"woman" + 0.029*"stand" + 0.026*"fight" + 0.024*"right"')
(12, '0.028*"people" + 0.026*"trump" + 0.022*"please" + 0.021*"donate"')
(13, '0.048*

### run on sampled data

In [20]:
dem_sampled_text_data = get_text_data(dem_sampled_tweets['stripped_text'].values)

0 / 94982
20000 / 94982
40000 / 94982
60000 / 94982
80000 / 94982
finished


In [21]:
topics = run_topic_model(text_data=dem_sampled_text_data
                         , num_topics=15
                         , num_passes=15
                         , num_words=4
                         , model_name=''
                        , save=False)

for topic in topics:
    print(topic)

(0, '0.043*"district" + 0.021*"congress" + 0.021*"people" + 0.019*"change"')
(1, '0.025*"supporter" + 0.019*"street" + 0.017*"north" + 0.015*"belief"')
(2, '0.469*"SCREEN_NAME" + 0.017*"thank" + 0.013*"proud" + 0.009*"happy"')
(3, '0.044*"woman" + 0.028*"health" + 0.025*"every" + 0.024*"fight"')
(4, '0.040*"trump" + 0.018*"president" + 0.018*"right" + 0.017*"country"')
(5, '0.043*"school" + 0.041*"student" + 0.041*"teacher" + 0.033*"education"')
(6, '0.047*"great" + 0.029*"SCREEN_NAME" + 0.024*"county" + 0.018*"candidate"')
(7, '0.095*"support" + 0.063*"thank" + 0.057*"campaign" + 0.028*"thanks"')
(8, '0.029*"congress" + 0.025*"sense" + 0.021*"common" + 0.016*"still"')
(9, '0.027*"endorsement" + 0.025*"worker" + 0.025*"receive" + 0.022*"union"')
(10, '0.020*"world" + 0.017*"people" + 0.015*"serve" + 0.013*"american"')
(11, '0.030*"family" + 0.019*"class" + 0.016*"security" + 0.015*"middle"')
(12, '0.029*"healthcare" + 0.019*"vote" + 0.017*"american" + 0.016*"democracy"')
(13, '0.023*"v

# Republican Topic Modeling

In [24]:
rep_tweets['stripped_text'] = rep_tweets['stripped_text'].astype(str)

In [25]:
rep_text_data = get_text_data(rep_tweets['stripped_text'].values)

0 / 169968
20000 / 169968
40000 / 169968
60000 / 169968
80000 / 169968
100000 / 169968
120000 / 169968
140000 / 169968
160000 / 169968
finished


In [26]:
topics = run_topic_model(text_data=rep_text_data
                         , num_topics=15
                         , num_passes=15
                         , num_words=4
                         , model_name='rep_model_15')

for topic in topics:
    print(topic)

(0, '0.036*"family" + 0.026*"america" + 0.023*"nation" + 0.021*"today"')
(1, '0.595*"SCREEN_NAME" + 0.017*"senate" + 0.013*"follow" + 0.009*"candidate"')
(2, '0.032*"candidate" + 0.027*"campaign" + 0.024*"election" + 0.021*"voter"')
(3, '0.020*"voting" + 0.020*"vote" + 0.018*"senator" + 0.017*"healthcare"')
(4, '0.046*"national" + 0.028*"security" + 0.024*"sign" + 0.021*"rally"')
(5, '0.058*"right" + 0.029*"debate" + 0.028*"freedom" + 0.028*"tonight"')
(6, '0.081*"great" + 0.074*"thank" + 0.045*"support" + 0.037*"county"')
(7, '0.057*"president" + 0.055*"trump" + 0.029*"congress" + 0.022*"conservative"')
(8, '0.035*"state" + 0.032*"business" + 0.029*"government" + 0.027*"people"')
(9, '0.039*"illegal" + 0.037*"border" + 0.034*"immigration" + 0.031*"problem"')
(10, '0.042*"liberal" + 0.028*"democrat" + 0.025*"group" + 0.025*"try"')
(11, '0.037*"would" + 0.023*"think" + 0.021*"politician" + 0.017*"career"')
(12, '0.051*"veteran" + 0.047*"forward" + 0.037*"happy" + 0.033*"looking"')
(13, 

## Look at tweets > 4 words

In [18]:
rep_long_tweets = rep_tweets[rep_tweets['stripped_text'].apply(lambda x: len(x.split(' ')) > 4)]
rep_text_data_long = get_text_data(rep_long_tweets['stripped_text'].values)


0 / 159538
20000 / 159538
40000 / 159538
60000 / 159538
80000 / 159538
100000 / 159538
120000 / 159538
140000 / 159538
finished


In [19]:
# BEST RESULTS

topics = run_topic_model(text_data=rep_text_data_long
                         , num_topics=15
                         , num_passes=15
                         , num_words=4
                         , model_name='rep_model_long_15')

for topic in topics:
    print(topic)

(0, '0.586*"SCREEN_NAME" + 0.015*"interview" + 0.015*"agriculture" + 0.014*"senate"')
(1, '0.084*"wisen" + 0.061*"great" + 0.057*"sendinthemarine" + 0.050*"wiright"')
(2, '0.042*"trump" + 0.041*"president" + 0.024*"economy" + 0.021*"senator"')
(3, '0.068*"business" + 0.025*"small" + 0.021*"young" + 0.020*"class"')
(4, '0.081*"fight" + 0.049*"conservative" + 0.043*"endorse" + 0.035*"veteran"')
(5, '0.066*"wyoming" + 0.040*"government" + 0.036*"governor" + 0.036*"state"')
(6, '0.034*"would" + 0.027*"baldwin" + 0.024*"need" + 0.024*"healthcare"')
(7, '0.092*"thank" + 0.058*"family" + 0.030*"today" + 0.026*"great"')
(8, '0.048*"protect" + 0.028*"never" + 0.028*"american" + 0.026*"always"')
(9, '0.034*"include" + 0.030*"enjoy" + 0.026*"christmas" + 0.025*"break"')
(10, '0.050*"school" + 0.029*"texas" + 0.021*"call" + 0.019*"first"')
(11, '0.053*"right" + 0.036*"woman" + 0.032*"forward" + 0.028*"looking"')
(12, '0.050*"going" + 0.038*"support" + 0.024*"effort" + 0.022*"future"')
(13, '0.076*

In [84]:
count = 0
print(len(rep_text_data_long))
for t in rep_text_data_long:
    if 'thank' in t:
        count += 1
print(count)

1628
152


## Run on sampled data
+ tweets that had > 4 words

In [22]:
rep_sampled_text_data = get_text_data(rep_sampled_tweets['stripped_text'].values)

0 / 54039
20000 / 54039
40000 / 54039
finished


In [28]:
# ALSO WORKED WELL

topics = run_topic_model(text_data=rep_sampled_text_data
                         , num_topics=15
                         , num_passes=15
                         , num_words=4
                         , model_name='rep_sampled_model'
                        , save=False)

for topic in topics:
    print(topic)

(0, '0.022*"liberal" + 0.020*"family" + 0.015*"prayer" + 0.013*"attack"')
(1, '0.025*"border" + 0.019*"school" + 0.017*"education" + 0.015*"student"')
(2, '0.013*"listen" + 0.012*"anyone" + 0.012*"pennsylvania" + 0.010*"watch"')
(3, '0.048*"candidate" + 0.032*"tennessee" + 0.031*"district" + 0.026*"republican"')
(4, '0.026*"working" + 0.019*"congress" + 0.018*"forward" + 0.016*"appreciate"')
(5, '0.028*"protect" + 0.017*"business" + 0.017*"right" + 0.013*"amendment"')
(6, '0.016*"honor" + 0.016*"people" + 0.016*"state" + 0.015*"veteran"')
(7, '0.339*"SCREEN_NAME" + 0.011*"senator" + 0.010*"state" + 0.010*"congress"')
(8, '0.050*"thank" + 0.036*"today" + 0.027*"great" + 0.017*"support"')
(9, '0.047*"great" + 0.043*"SCREEN_NAME" + 0.031*"county" + 0.018*"thanks"')
(10, '0.023*"medium" + 0.013*"committee" + 0.012*"american" + 0.011*"festival"')
(11, '0.021*"would" + 0.019*"record" + 0.014*"state" + 0.013*"first"')
(12, '0.041*"happy" + 0.020*"family" + 0.018*"christmas" + 0.017*"dinner"')